In [0]:
###출처: https://github.com/2alive3s/Fake_news

import tensorflow as tf
import numpy as np
#from text_cnn import TextCNN
from tensorflow.contrib.rnn import GRUCell
from tensorflow.python.ops.rnn import bidirectional_dynamic_rnn as bi_rnn #brnn layer를 위해
#from tensorflow.contrib.rnn import stack_bidirectional_rnn as bi_rnn


class Affine(object):
# Combine all the pooled features
    def __init__( #초기화 메서드(새로운 인스턴스 만들어질때 호출)
      self, sequence_length_head, sequence_length_body, num_classes, vocab_size_head, vocab_size_body,
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.1): #embedding_size:embedding vector크기
        
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")     #예측출력갯수
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")      #dropout
        self.input_x_head = tf.placeholder(tf.int32, [None, sequence_length_head], name="input_x_head")  #input x의 헤더
        self.input_x_body = tf.placeholder(tf.int32, [None, sequence_length_body], name="input_x_body")  #input x의 바디
        
        # Embedding layer
        self.embeddings_head = tf.Variable(
                tf.random_uniform([vocab_size_head, embedding_size], -1.0, 1.0),trainable=False)#trainable=false #모델 결과 값인 임베딩 벡터를 저장할 변수
                                                                                                #총 단어 갯수와 임베딩 갯수를 크기로 하는 두개의 차원을 가짐 
        self.embedded_chars_head = tf.nn.embedding_lookup(self.embeddings_head, self.input_x_head) #큰사이즈의 리스트에 담겨진 데이터를 입력받은 인덱스에 따라 look up
                                                                                                   #embeddings_head와 input_x_head가 처리할 인덱스
                                                                                                   # 임베딩 벡터의 차원에서 학습할 입력값에 대한 행들을 뽑음
                                                                                                   # 예) embeddings     inputs    selected
                                                                                                   #    [[1, 2, 3]  -> [2, 3] -> [[2, 3, 4]
                                                                                                   #     [2, 3, 4]                [3, 4, 5]]
                                                                                                   #     [3, 4, 5]
                                                                                                   #     [4, 5, 6]]
        self.embedded_chars_expanded_head = tf.expand_dims(self.embedded_chars_head, -1)

        self.embeddings_body = tf.Variable(
                tf.random_uniform([vocab_size_body, embedding_size], -1.0, 1.0),trainable=False)#trainable=false
        self.embedded_chars_body = tf.nn.embedding_lookup(self.embeddings_body, self.input_x_body)
        self.embedded_chars_expanded_body = tf.expand_dims(self.embedded_chars_body, -1) #expand_dims: 크기가 1인차원을 텐서의 구조에 삽입
                                                                                         #음수가 지정되었으므로, 끝에서부터 역으로 계산
        
        #2. LSTM LAYER ######################################################################
        with tf.variable_scope("lstm-head") as scope: #lstm-head/변수이름 // 변수이름충돌방지
            #self.lstm_cell_head = tf.contrib.rnn.LSTMCell(embedding_size,state_is_tuple=True)
            #self.lstm_out_head,self.lstm_state_head = tf.nn.dynamic_rnn(self.lstm_cell_head,self.embedded_chars_head,dtype=tf.float32)
            #self.lstm_out_expanded_head = tf.expand_dims(self.lstm_out_head, -1)
            self.lstm_out_head,self.lstm_state_head = bi_rnn(GRUCell(embedding_size), GRUCell(embedding_size), inputs = self.embedded_chars_head , dtype=tf.float32) #gru셀 계산
            self.lstm_out_merge_head = tf.concat(self.lstm_out_head, axis=2) #lst_out_head를 2번째차원을 제거후 붙이겠다
            #self.lstm_out_head_fw = self.lstm_out_head[0]
            #self.lstm_out_head_bw = self.lstm_out_head[1]
            #self.lstm_out_merge_head = tf.concat([self.lstm_out_head_fw[-1], self.lstm_out_head_bw[-1]], axis=1)
            self.lstm_out_expanded_head = tf.expand_dims(self.lstm_out_merge_head, -1) #크기가 1인차원 끝에 삽입
            print(self.lstm_out_expanded_head.shape)

#output = tf.stack(output, axis=1)
#output = tf.reshape(output, [-1, FLAGS.num_units * 2])

        with tf.variable_scope("lstm-body") as scope:
            #self.lstm_cell_body = tf.contrib.rnn.LSTMCell(embedding_size,state_is_tuple=True)
            #self.lstm_out_body,self.lstm_state_body = tf.nn.dynamic_rnn(self.lstm_cell_body,self.embedded_chars_body,dtype=tf.float32)
            #self.lstm_out_expanded_body = tf.expand_dims(self.lstm_out_body, -1)
            self.lstm_out_body,self.lstm_state_body = bi_rnn(GRUCell(embedding_size), GRUCell(embedding_size), inputs = self.embedded_chars_body , dtype=tf.float32)
            self.lstm_out_merge_body = tf.concat(self.lstm_out_body, axis=2)
            #self.lstm_out_body_fw = self.lstm_out_body[0]
            #self.lstm_out_body_bw = self.lstm_out_body[1]
            #self.lstm_out_merge_body = tf.concat([self.lstm_out_body_fw[-1], self.lstm_out_body_bw[-1]], axis=1)
            self.lstm_out_expanded_body = tf.expand_dims(self.lstm_out_merge_body, -1)
            
            print(self.lstm_out_expanded_body.shape)
        
        self.pooled_outputs_head = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-head-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size*2, 1, 256] #256의 컨볼루션 값=> 256으로 늘리겠다
                W_head = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W_head")
                b_head = tf.Variable(tf.constant(0.1, shape=[256]), name="b_head")
                conv_head = tf.nn.conv2d(
                    self.lstm_out_expanded_head,
                    W_head,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h_head = tf.nn.relu(tf.nn.bias_add(conv_head, b_head), name="relu_head")
                # Maxpooling over the outputs
                pooled_head = tf.nn.max_pool(
                    h_head,
                    ksize=[1, sequence_length_head - filter_size + 1, 1, 1], #커널사이즈 sequence_length_head-filter_size+1,1 #왜 이렇게 바꾸는지 알아보기!??
                    strides=[1, 1, 1, 1],                                    #max_pooling 적용후 텐서의 차원 수 [batch_size,1,1num_filters]
                    padding='VALID',
                    name="pool")
                self.pooled_outputs_head.append(pooled_head) #pooled_ouputs_head의 풀링거친 첫번째 컨볼루션레이어값 추가

        self.pooled_outputs_body = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-body-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size*2, 1, 1024] 
                W_body = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W_body")
                b_body = tf.Variable(tf.constant(0.1, shape=[1024]), name="b_body")
                conv_body = tf.nn.conv2d(
                    self.lstm_out_expanded_body,
                    W_body,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h_body = tf.nn.relu(tf.nn.bias_add(conv_body, b_body), name="relu_body")
                # Maxpooling over the outputs
                pooled_body = tf.nn.max_pool(
                    h_body,
                    ksize=[1, sequence_length_body - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                self.pooled_outputs_body.append(pooled_body)        
        
        l2_loss = tf.constant(0.0) #0으로 설정
        
        pooled_outputs = tf.concat([self.pooled_outputs_head,self.pooled_outputs_body],-1,name='preconcat') #value(-1)차원으로 이어 붙이기
        print(pooled_outputs.shape)
        num_filters_total = num_filters * len(filter_sizes) #토탈필터사이즈구하기
        self.h_pool = tf.concat(pooled_outputs, 3, name='concat')  #pooled_outputs을 3번째차원을 제거후 붙이겠다
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total]) #fully connected layer로 넣기 위해 펼치기
       
       	######fully connected layer
        W_fc1 = tf.Variable(tf.truncated_normal([1280,1024],stddev=0.1),name="W_fc1") 
        b_fc1 = tf.Variable(tf.constant(0.1,shape=[1024]),name="b_fc1")
        h_fc1 = tf.nn.relu(tf.matmul(self.h_pool_flat,W_fc1) + b_fc1) #relu사용
        
        W_fc2 = tf.Variable(tf.truncated_normal([1024,1024],stddev=0.1),name="W_fc1")
        b_fc2 = tf.Variable(tf.constant(0.1,shape=[1024]),name="b_fc1")
        h_fc2 = tf.nn.relu(tf.matmul(h_fc1,W_fc2) + b_fc2)
        
        W_fc3 = tf.Variable(tf.truncated_normal([1024,1024],stddev=0.1),name="W_fc1")
        b_fc3 = tf.Variable(tf.constant(0.1,shape=[1024]),name="b_fc1") #ouput 전 1024
        h_fc3 = tf.nn.relu(tf.matmul(h_fc2,W_fc3) + b_fc3)
            
        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(h_fc3, self.dropout_keep_prob) #마지막 h_fc3만 dropout?//그럼 중간중간 droupout 추가하면 정확도증가???(시도!)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[1024, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b") #마지막 output shape로 num_classes까지 줄이기
            l2_loss += tf.nn.l2_loss(W) #tf.nn.l2_loss는 전달받은 텐서의 요소들의 제곱의 합을 2로 나누어 리턴
            l2_loss += tf.nn.l2_loss(b) #w와 b를ㅣ2_loss로 리턴한후 그값들을 l2_loss변수에 추가
            self.scores = tf.nn.xw_plus_b(self.h_drop, self.W, self.b, name="scores") #Computes matmul(x, weights) + biases
            self.probabilities = tf.nn.softmax(self.scores)
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # CalculateMean cross-entropy loss
        with tf.name_scope("loss"): #손실함수 정의
            print(self.scores.shape)
            losses = tf.nn.softmax_cross_entropy_with_logits(logits = self.scores, labels = self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            print("%d/%d",self.predictions,self.input_y)
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [16]:
# -*- coding: utf-8 -*-:
"""
Created on Wed Nov  8 23:58:05 2017
@author: samsung
"""
!apt-get update
!apt-get install g++ openjdk-8-jdk
!pip3 install konlpy

import numpy as np
import codecs
import re
import itertools
from collections import Counter
from konlpy.tag import Komoran
from csv import DictReader
from csv import DictWriter

pos_tagger = Komoran()

class Data:
    def __init__(self, file_instances):
        # Load data
        self.instances = self.read(file_instances)
        self.headlines = {}
        self.bodies = {}
        self.labels = {}
         
        for instance in self.instances:
            instance['seqid'] = int(instance['seqid'])
            
        for head in self.instances:
            self.headlines[head['seqid']] = head['title']
        
        for body in self.instances:
            self.bodies[body['seqid']] = body['content']
        
        for label in self.instances:
            self.labels[label['seqid']] = label['Label']
                
    def read(self, filename):
        
        rows = []
        # Process file
        with open(filename, "r") as table:
            r = DictReader(table)
            for line in r:
                rows.append(line)
        return rows
    
    def get_data(self):
        return self.instances
        
def tokenize(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    final_string = ''
    tokenized = [''.join(t) for t in pos_tagger.pos(string) if t[1] in ['NNG','NNP','NNB','NR','VV','VA','VCP','VCN','XSV','XSA','SN','MAG','MM','MAJ']]
#not in ['ETM','EC','ETN','EF','EP','NF','NV','NA','SW','SO','SP','SF','SE','SS','IC','XSN','XPN']]
    for tokens in tokenized:
        final_string += tokens + " "
    return final_string

def flat(content):
    return ["{}{}".format(word, tag) for word, tag in pos_tagger.pos(content)]


def load_data_and_labels(file_instances):
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load data from files

    train_heads = []
    train_bodies = []
    train_labels = []
    data = Data(file_instances)
    
    for instance in data.instances:
        news_id = instance['seqid']
        train_label = instance['Label']
        train_heads.append(tokenize(data.headlines[news_id]))
        train_bodies.append(tokenize(data.bodies[news_id]))
        train_labels.append((news_id,train_label))

    results = np.zeros((len(train_labels),2))

    for i, train_label in train_labels:
        if train_label == '1':
            results[i,1] = 1
        else:
            results[i,0] = 1

    return train_heads, train_bodies, results


def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int(len(data)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

def load_word_embedding(file_name, vocab_processor, embedding_dim):

    initW = np.random.uniform(-0.25, 0.25, (len(vocab_processor.vocabulary_), embedding_dim))
    print("Load word2vec file {}\n".format(file_name))
    with open(file_name, "rb") as f:
        for idx, line in enumerate(f):
            word = []
            vectors = []
            if idx == 0:
                vocab_size, dim = line.strip().split()
            else:
                tks = line.strip().split()
                word = tks[0].strip().decode('utf8')
                vectors = tks[1:]
                idx = vocab_processor.vocabulary_.get(word)
                if idx != 0:
                    initW[idx] = np.array(vectors)
    print(initW)
    return initW

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [564 B]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [801 B]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [320 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic

In [15]:
from google.colab import drive
import os

if os.path.exists('/content/gdrive')==False:
  drive.mount('/content/gdrive')
  print('Google Drive is mounted\n')
else:
  print('Google Drive is already mounted\n')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
Google Drive is mounted



In [22]:
# -*- coding: utf-8 -*-
"""
Created on Wed Nov  8 23:58:50 2017
@author: samsung
"""

#! /usr/bin/env python

import tensorflow as tf
import numpy as np
import os
import time
import datetime
#import data_helpers
#from affine import Affine
from tensorflow.contrib import learn
import codecs
import operator
from pandas.io.parsers import read_csv

'''
# Parameters
# ==================================================
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim",128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 1280, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.1, "L2 regularizaion lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size",127, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 100, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")
'''
embedding_dim=128
filter_sizes="3"
num_filters=1280
dropout_keep_prob=0.5
l2_reg_lambda=0.1 
batch_size=127,
num_epochs=100
evaluate_every=100
checkpoint_every=100
allow_soft_placement= True
log_device_placement= False


# Data Preparatopn
# ==================================================




model = tf.global_variables_initializer()

if os.path.exists('/content/gdrive/My Drive/Colab Notebooks/mission1_test.csv')==False:
  print('Train data downloading..')
  ! curl 'https://raw.githubusercontent.com/2alive3s/Fake_news/master/data/mission1_test.csv' -o '/content/gdrive/My Drive/Colab Notebooks/mission1_test.csv'
  print('Done..\n')
else:
  print('File already exists \n')
    
# 학습데이터 로딩
file_train_instances = read_csv('/content/gdrive/My Drive/Colab Notebooks/mission1_test.csv')

# Load data
print("Loading data...")
x_heads, x_bodies, y = data_helpers.load_data_and_labels(file_train_instances)

#bow_head = TfidfVectorizer(tokenizer=None, lowercase=False, max_features=128)
#bow_body = TfidfVectorizer(tokenizer=None, lowercase=False, max_features=1280)

#bow_head_vec = bow_head.fit_transform(x_heads)
#bow_body_vec = bow_body.fit_transform(x_bodies)

#bow_head_vocab = [v[0] for v in sorted(bow_head.vocabulary_.items(),key=operator.itemgetter(1))]
#print(x_heads)
#print(bow_head_vocab)

# Build vocabulary_head
max_document_length_head = max([len(x.split(" ")) for x in x_heads])
vocab_processor_head = learn.preprocessing.VocabularyProcessor(max_document_length=128)#bow_head max_features
x_head = np.array(list(vocab_processor_head.fit_transform(x_heads)))
# Build vocabulary_body
max_document_length_body = max([len(x.split(" ")) for x in x_bodies])
vocab_processor_body = learn.preprocessing.VocabularyProcessor(max_document_length=1280)
x_body = np.array(list(vocab_processor_body.fit_transform(x_bodies)))

print('----headline_shape----')
print(x_head.shape)
print('----body_shape----')
print(x_body.shape)
print('----label_shape----')
print(y.shape)

# Randomly shuffle data
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))

x_head_shuffled = x_head[shuffle_indices]
x_body_shuffled = x_body[shuffle_indices]
y_shuffled = y[shuffle_indices]

# Split train/test set
# TODO: This is very crude, should use cross-validation
x_train_head, x_dev_head = x_head_shuffled[:-6000], x_head_shuffled[-6000:]
x_train_body, x_dev_body = x_body_shuffled[:-6000], x_body_shuffled[-6000:]
y_train, y_dev = y_shuffled[:-6000], y_shuffled[-6000:]
print("Vocabulary Size_head: {:d}".format(len(vocab_processor_head.vocabulary_)))
print("Vocabulary Size_body: {:d}".format(len(vocab_processor_body.vocabulary_)))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))
print(x_train_head)
print(x_train_body)
print(x_train_head.shape)
print(x_train_body.shape)
# Training
# ==================================================

with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=allow_soft_placement,
      log_device_placement=log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = Affine(
            sequence_length_head=128,
            sequence_length_body=1280,
            num_classes=2,
            vocab_size_head=len(vocab_processor_head.vocabulary_),
            vocab_size_body=len(vocab_processor_body.vocabulary_),
            embedding_size=embedding_dim,
            filter_sizes=list(map(int,filter_sizes.split(","))),
            num_filters=num_filters,
            l2_reg_lambda=l2_reg_lambda
            )

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(),max_to_keep=500)

        # Write vocabulary
        vocab_processor_head.save(os.path.join(out_dir, "vocab_head"))
        vocab_processor_body.save(os.path.join(out_dir, "vocab_body"))
        
        # Initialize all variables
        sess.run(tf.global_variables_initializer())
        head_embedding = data_helpers.load_word_embedding("fasttext_3_10.vec",vocab_processor_head,embedding_dim)
        body_embedding = data_helpers.load_word_embedding("fasttext_3_10.vec",vocab_processor_body,embedding_dim)
        sess.run(cnn.embeddings_head.assign(head_embedding))
        sess.run(cnn.embeddings_body.assign(body_embedding))
 #       sess.run(cnn.cnn_head.W.assign(initW))
 #       sess.run(cnn.cnn_body.W.assign(initW))
            
        def train_step(x_batch_head, x_batch_body, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x_head: x_batch_head,
              cnn.input_x_body: x_batch_body,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: dropout_keep_prob
            }
            _, step, loss, accuracy, predictions = sess.run(
                [train_op, global_step, cnn.loss, cnn.accuracy, cnn.predictions],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            print(predictions)
            #train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch_head, x_batch_body, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x_head: x_batch_head,
              cnn.input_x_body: x_batch_body,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, loss, accuracy, predictions = sess.run(
                [global_step, cnn.loss, cnn.accuracy, cnn.predictions],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            return accuracy, loss, predictions
            #if writer:
            #    writer.add_summary(summaries, step)

        # Generate batches
        batches = data_helpers.batch_iter(
            list(zip(x_train_head, x_train_body, y_train)), batch_size, num_epochs)
      
        # Training loop. For each batch...
        for batch in batches:
            x_batch_head, x_batch_body, y_batch = zip(*batch)
            train_step(x_batch_head, x_batch_body, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % evaluate_every == 0:
                print("\nEvaluation:")
                batches_dev = data_helpers.batch_iter(list(zip(x_dev_head, x_dev_body, y_dev)),31, 1)
                acc_total, loss_total, total = 0,0,0
                for batch_dev in batches_dev:
                    x_batch_dev_head, x_batch_dev_body, y_batch_dev = zip(*batch_dev)
                    acc_dev, loss_dev, predictions = dev_step(x_batch_dev_head, x_batch_dev_body, y_batch_dev)
                    acc_total = acc_total + acc_dev
                    loss_total = loss_total + loss_dev
                    print(predictions)
                    total = total + 1
                results = codecs.open('results_all_new.txt','a')
                result = "total loss {:g}, total acc {:g}".format(loss_total/total, acc_total/total)
                print(result)
                results.write("step{:g}, total loss{:g}, total acc{:g}".format(current_step, loss_total/total, acc_total/total) + '\n')
                results.close()
                print("")
            if current_step % checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print(path)
                print("Saved model checkpoint to {}\n".format(path))

File already exists 



ParserError: ignored